# dlt Homework

## Q1 dlt Version
In this homework, we will load the data from our FAQ to Qdrant<br>
Let's install dlt with Qdrant support and Qdrant client

In [ ]:
# !pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"

### dlt Resource

In [1]:
import dlt
import requests

@dlt.resource()
def zoomcamp_data():
    """
    This generator function fetches course documents from a URL,
    adds the course name to each document, and yields them one by one.
    The @dlt.resource decorator marks this function as a data source for dlt.
    """
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']
        
        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

In [3]:
print(f'dlt version: {dlt.__version__}')

dlt version: 1.13.0


## Q2. dlt pipeline

In [4]:
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

# Run the pipeline
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())

print("Pipeline run finished!")
print(f'\nPrinting load info:\n{load_info}\n\n')
print(f'\nPrinting last trace:\n{pipeline.last_trace}')

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

Pipeline run finished!

Printing load info:
Pipeline zoomcamp_pipeline load step completed in 4.05 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /workspaces/llm-zoomcamp/workshops/db.qdrant location to store data
Load package 1752148609.1325104 is LOADED and contains no failed jobs



Printing last trace:
Run started at 2025-07-10 11:56:43.138773+00:00 and COMPLETED in 10.49 seconds with 4 steps.
Step extract COMPLETED in 0.35 seconds.

Load package 1752148609.1325104 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.09 seconds.
Normalized data for the following tables:
- _dlt_pipeline_state: 1 row(s)
- zoomcamp_data: 948 row(s)

Load package 1752148609.1325104 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 4.07 seconds.
Pipeline zoomcamp_pipeline load step completed in 4.05 seconds
1 load pa

In [5]:
print('Number of rows for zoomcamp data is 948 rows')

Number of rows for zoomcamp data is 948 rows


# Q3. Embeddings
The meta.json files shows the fast-bge-small-en model was used to embed the FAQs